Lecture 19: Sparse Autoencoders for MNIST classification
==

## Load Packages

In [ ]:
%matplotlib inline
import copy
import torch
import numpy as np
import torchvision
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torchvision.transforms as transforms

print(torch.__version__) # This code has been updated for PyTorch 1.0.0

## Load Data:

In [ ]:
transform = transforms.Compose([transforms.ToTensor()])
BatchSize = 1000

trainset = torchvision.datasets.MNIST(root='./MNIST', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BatchSize,
                                          shuffle=True, num_workers=4) # Creating dataloader

testset = torchvision.datasets.MNIST(root='./MNIST', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BatchSize,
                                         shuffle=False, num_workers=4) # Creating dataloader

classes = ('zero', 'one', 'two', 'three',
           'four', 'five', 'six', 'seven', 'eight', 'nine')

In [ ]:
# Check availability of GPU

use_gpu = torch.cuda.is_available()
if use_gpu:
    print('GPU is available!')
    device = "cuda"
else:
    print('GPU is not available!')
    device = "cpu"

## Define the Sparse Autoencoder:

In [ ]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28*28, 400),
            nn.Tanh())
        self.decoder = nn.Sequential(
            nn.Linear(400, 28*28),
            nn.Sigmoid())

    def forward(self, x):
        x = self.encoder(x)        
        x = self.decoder(x)
        return x


net = autoencoder()
print(net)

net = net.to(device)

init_weightsE = copy.deepcopy(net.encoder[0].weight.data)
init_weightsD = copy.deepcopy(net.decoder[0].weight.data)

## Train Autoencoder:

In [ ]:
iterations = 20 # >10
learning_rate = 10
lambda1 = 0.1 # sparsity factor
criterion = nn.MSELoss()

dataiter = iter(testloader)
test_images, test_labels = dataiter.next()
printImg = torch.Tensor(int(iterations/10),2,1,28,28)

for epoch in range(iterations):  # loop over the dataset multiple times
    runningLoss = 0.0
    net.train()
    for data in trainloader:
        # get the inputs
        inputs, labels = data
        inputs = inputs.view(-1, 28*28).to(device)
        net.zero_grad()  # zeroes the gradient buffers of all parameters
        outputs = net(inputs) # forward 
        loss = criterion(outputs, inputs) # calculate loss
        l1_norm = lambda1*torch.norm(net.encoder[0].weight, p=1) # L1 penalty for the encoder
        total_loss = loss+l1_norm        
        total_loss.backward() #  backpropagate the loss
        for f in net.parameters():
            f.data.sub_(f.grad.data * learning_rate) # weight = weight - learning_rate * gradient (Update Weights)
        runningLoss += loss.item()
        
    print('At Iteration : %d / %d  ;  Mean-Squared Error : %f'%(epoch + 1,iterations,
                                                                        runningLoss/(60000/BatchSize)))
    
    if epoch%10 == 0:
        net.eval()
        with torch.no_grad():
            outImg = net((test_images[0].view(-1,28*28).to(device))).data
            outImg = outImg.view(-1,28,28)
            if use_gpu:
                outImg = outImg.cpu()   

            dispImg = torch.Tensor(2,1,28,28)
            dispImg[0] = test_images[0]
            dispImg[1] = outImg
            printImg[int(epoch/10)] = dispImg
print('Finished Training')

## Autoencoder Performance:

In [ ]:
# functions to show an image
def imshow(img, strlabel):
    npimg = img.numpy()
    npimg = np.abs(npimg)
    fig_size = plt.rcParams["figure.figsize"]
    fig_size[0] = 10
    fig_size[1] = 10
    plt.rcParams["figure.figsize"] = fig_size
    plt.figure()
    plt.title(strlabel)
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

In [ ]:
for i in range(int(iterations/10)):
    imshow(torchvision.utils.make_grid(printImg[i]), 'Autoencoder Performance')

## Encoder Weight Visualisation:

In [ ]:
trained_weightsE = copy.deepcopy(net.encoder[0].weight.data)
d_weightsE = init_weightsE - trained_weightsE 

init_weightsE = init_weightsE.view(400,1,28,28)
trained_weightsE = trained_weightsE.view(400,1,28,28)
d_weightsE = d_weightsE.view(400,1,28,28)
if use_gpu:
    init_weightsE = init_weightsE.cpu()
    trained_weightsE = trained_weightsE.cpu()
    d_weightsE = d_weightsE.cpu()  

imshow(torchvision.utils.make_grid(init_weightsE,nrow=20,normalize=True),'Initial Weights')
imshow(torchvision.utils.make_grid(trained_weightsE,nrow=20,normalize=True),'Trained Weights')
imshow(torchvision.utils.make_grid(d_weightsE,nrow=20,normalize=True), 'Weight update')

## Decoder Weight Visualisation:

In [ ]:
trained_weightsD = copy.deepcopy(net.decoder[0].weight.data)
d_weightsD = init_weightsD - trained_weightsD 

init_weightsD = init_weightsD.view(784,1,20,20)
trained_weightsD = trained_weightsD.view(784,1,20,20)
d_weightsD = d_weightsD.view(784,1,20,20)
if use_gpu:
    init_weightsD = init_weightsD.cpu()
    trained_weightsD = trained_weightsD.cpu()
    d_weightsD = d_weightsD.cpu()

imshow(torchvision.utils.make_grid(init_weightsD,nrow=28,normalize=True),'Initial Weights')
imshow(torchvision.utils.make_grid(trained_weightsD,nrow=28,normalize=True),'Trained Weights')
imshow(torchvision.utils.make_grid(d_weightsD,nrow=28,normalize=True), 'Weight update')

## Modifying the autoencoder for classification: 

In [ ]:
# Removing the decoder module from the autoencoder
new_classifier = nn.Sequential(*list(net.children())[:-1])
net = new_classifier
# Adding linear layer for 10-class classification problem
net.add_module('classifier', nn.Sequential(nn.Linear(400, 10),nn.LogSoftmax(dim=1)))
print(net)
net = net.to(device)

## Train Classifier:

In [ ]:
iterations = 20
learning_rate = 0.1
criterion = nn.NLLLoss()

for epoch in range(iterations):  # loop over the dataset multiple times

    runningLoss = 0.0
    net.train()
    for data in trainloader:
        # get the inputs
        inputs, labels = data
        
        inputs, labels = inputs.view(-1, 28*28).to(device), labels.to(device)

        net.zero_grad()  # zeroes the gradient buffers of all parameters
        outputs = net(inputs) # forward 
        loss = criterion(outputs, labels) # calculate loss
        loss.backward() #  backpropagate the loss
        for f in net.parameters():
            f.data.sub_(f.grad.data * learning_rate) # weight = weight - learning_rate * gradient (Update Weights)
        runningLoss += loss.item()
        correct = 0
        total = 0
    net.eval()
    with torch.no_grad():
        for data in testloader:
            inputs, labels = data
            inputs, labels = inputs.view(-1, 28*28).to(device), labels.to(device)
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum()
    print('At Iteration : %d / %d  ;  Train Error : %f ;Test Accuracy : %f'%(epoch + 1,iterations,
                                                                        runningLoss/(60000/BatchSize),100 * float(correct) /float(total)))
print('Finished Training')

## Performance of different Classes:

In [ ]:
class_correct = np.zeros(10)
class_total = np.zeros(10)
net.eval()

with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images.view(-1, 28*28).to(device))
        _, predicted = torch.max(outputs.data, 1)
        if use_gpu:
            predicted = predicted.cpu()        
        c = (predicted == labels).squeeze(0)        
        for i in range(images.shape[0]):
            label = labels[i]                            
            class_correct[label] += float(c[i])
            class_total[label] += 1

for i in range(10):
    print('Accuracy of %5s : %f %%' % (
        classes[i], 100 * float(class_correct[i]) / float(class_total[i])))